In [2]:
import os, glob, gc, copy, math, random, time, json
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
from tqdm import tqdm

# ══════════════════════════ KONFIGURASI ══════════════════════════
DATA_DIR    = "/workspace/SPLIT_BEATS_NPY/train"
LABEL_MAP   = {'N':0,'L':1,'R':2,'V':3,'Q':4}
SEED        = 42
N_SPLITS    = 5
EPOCHS      = 1
BATCH_SIZE  = 16
MAX_LEN     = 512
EMB_DIM     = 512
N_HEADS     = 8
FF_DIM      = 2048
N_LAYERS    = 12
LR          = 2e-5
OUTPUT_BASE = "/workspace/HASIL_DECODER/HASIL_1"
os.makedirs(OUTPUT_BASE, exist_ok=True)

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_ID, CLS_ID  = 256, 257
VOCAB_SIZE      = 258
cls_names       = list(LABEL_MAP.keys())

# ══════════ UTILITAS ══════════

def signal_to_ids(sig: np.ndarray):
    norm = ((sig - sig.min()) / (sig.ptp() + 1e-8) * 255).astype(int)
    ids  = np.concatenate(([CLS_ID], norm))[:MAX_LEN]
    mask = np.ones_like(ids, dtype=int)
    if len(ids) < MAX_LEN:
        pad_len = MAX_LEN - len(ids)
        ids  = np.concatenate((ids,  np.full(pad_len, PAD_ID)))
        mask = np.concatenate((mask, np.zeros(pad_len)))
    return ids, mask

# ══════════ MUAT DATA ══════════
files, labels = [], []
for cls, idx in LABEL_MAP.items():
    for f in glob.glob(os.path.join(DATA_DIR, cls, "*.npy")):
        files.append(f); labels.append(idx)
files, labels = np.array(files), np.array(labels)

all_ids, all_mask = [], []
for f in tqdm(files, desc="Pre-encoding"):
    ids, msk = signal_to_ids(np.load(f))
    all_ids.append(ids);  all_mask.append(msk)
all_ids  = torch.tensor(all_ids,  dtype=torch.long)
all_mask = torch.tensor(all_mask, dtype=torch.long)
labels_t = torch.tensor(labels,   dtype=torch.long)

# ══════════ DEFINISI DECODER-ONLY ══════════
class DecoderOnlyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM, padding_idx=PAD_ID)
        self.pos_emb   = nn.Parameter(torch.zeros(1, MAX_LEN, EMB_DIM))
        dec_layer = nn.TransformerDecoderLayer(d_model=EMB_DIM, nhead=N_HEADS,
                                               dim_feedforward=FF_DIM, dropout=0.1,
                                               batch_first=True)
        self.decoder = nn.TransformerDecoder(dec_layer, num_layers=N_LAYERS)
        self.fc      = nn.Linear(EMB_DIM, len(LABEL_MAP))
        nn.init.normal_(self.pos_emb, std=0.02)

    def forward(self, input_ids, attention_mask):
        tgt = self.token_emb(input_ids) + self.pos_emb
        memory = torch.zeros_like(tgt).to(tgt.device)  # dummy memory
        x = self.decoder(tgt, memory, tgt_key_padding_mask=~attention_mask.bool())
        x = (x * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1, keepdim=True).clamp(min=1e-9)
        return self.fc(x)

# ══════════ K-FOLD TRAINING ══════════
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
rows_all = []

for fold, (tr, va) in enumerate(skf.split(all_ids, labels), 1):
    print(f"\n==== FOLD {fold}/{N_SPLITS} ====")
    gc.collect(); torch.cuda.empty_cache()

    train_ds = TensorDataset(all_ids[tr], all_mask[tr], labels_t[tr])
    val_ds   = TensorDataset(all_ids[va], all_mask[va], labels_t[va])
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    model = DecoderOnlyClassifier().to(DEVICE)
    optim = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, EPOCHS)

    best_state, best_loss = None, math.inf
    out_dir = os.path.join(OUTPUT_BASE, f"fold{fold}"); os.makedirs(out_dir, exist_ok=True)

    for epoch in range(1, EPOCHS + 1):
        print(f"\U0001f552 Fold {fold} | Epoch {epoch}/{EPOCHS}")
        model.train(); total_loss = 0.0
        for ids, msk, lbl in tqdm(train_loader, leave=False):
            ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
            optim.zero_grad(); out = model(ids, msk)
            loss = F.cross_entropy(out, lbl)
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step(); total_loss += loss.item()
        scheduler.step()

        model.eval(); val_loss, preds, yh = 0.0, [], []
        with torch.no_grad():
            for ids, msk, lbl in val_loader:
                ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
                out = model(ids, msk)
                val_loss += F.cross_entropy(out, lbl, reduction='sum').item()
                preds.append(out.argmax(1).cpu()); yh.append(lbl.cpu())
        val_loss /= len(val_ds)
        if val_loss < best_loss:
            best_loss, best_state = val_loss, copy.deepcopy(model.state_dict())
        print(f"ValLoss={val_loss:.4f}")

    # Simpan model dan config
    torch.save(best_state, os.path.join(out_dir, "best_model.pt"))
    with open(os.path.join(out_dir, "model_config.json"), "w") as f:
        json.dump({"emb_dim":EMB_DIM,"n_layers":N_LAYERS,"n_heads":N_HEADS,"ff_dim":FF_DIM,
                   "max_len":MAX_LEN,"vocab_size":VOCAB_SIZE,"pad_id":PAD_ID,"cls_id":CLS_ID,
                   "label_map":LABEL_MAP}, f, indent=2)
    with open(os.path.join(out_dir, "vocab.txt"), "w") as f:
        f.writelines([f"{i}\n" for i in range(256)] + ["[PAD]\n","[CLS]\n"])

    # Evaluasi dan confusion matrix
    model.load_state_dict(best_state); model.eval()
    with torch.no_grad():
        logits = []; y_true = []
        for ids, msk, lbl in val_loader:
            ids, msk = ids.to(DEVICE), msk.to(DEVICE)
            logits.append(model(ids, msk).cpu()); y_true.append(lbl)
    preds = torch.cat(logits).argmax(1).numpy(); y_true = torch.cat(y_true).numpy()

    cm = confusion_matrix(y_true, preds, labels=list(range(len(cls_names))))
    for i, cls in enumerate(cls_names):
        TP = cm[i,i]; FN = cm[i].sum()-TP; FP = cm[:,i].sum()-TP; TN = cm.sum()-TP-FN-FP
        ACC = (TP+TN)/cm.sum(); REC = TP/(TP+FN+1e-8)
        SPEC = TN/(TN+FP+1e-8); F1 = 2*TP/(2*TP+FP+FN+1e-8)
        rows_all.append({"fold":fold, "kelas":cls, "akurasi":round(ACC,4),
                         "f1":round(F1,4), "recall":round(REC,4), "spesifisitas":round(SPEC,4)})

    # Simpan confusion matrix gambar
    plt.figure(figsize=(7,6)); plt.imshow(cm, cmap='Blues')
    plt.title(f"Confusion Fold {fold}")
    plt.xticks(range(len(cls_names)), cls_names)
    plt.yticks(range(len(cls_names)), cls_names)
    for r in range(len(cm)):
        for c in range(len(cm)):
            plt.text(c, r, cm[r,c], ha='center', va='center')
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "confusion_fold.png"))
    plt.close()

# Simpan semua metrik
pd.DataFrame(rows_all).to_csv(os.path.join(OUTPUT_BASE, "final_summary_perkelas.csv"), index=False)
print("\n✔️ Training selesai. Semua hasil disimpan.")


Pre-encoding: 100%|██████████| 28000/28000 [00:03<00:00, 7486.37it/s]



==== FOLD 1/5 ====


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 23.58 GiB of which 16.44 MiB is free. Process 71901 has 10.64 GiB memory in use. Process 236872 has 8.58 GiB memory in use. Process 309636 has 4.33 GiB memory in use. Of the allocated memory 4.02 GiB is allocated by PyTorch, and 15.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF